In [ ]:
# @title Load a drive in in the notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Import necessary libraries and dependencies
import tensorflow
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Input, concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Model
from sklearn.model_selection import RepeatedKFold
from os import listdir
from os.path import isfile, join
import sys
import cv2
import math
import os
import numpy as np
MainFolder ='/content/drive/MyDrive/Office/domain_adaptation_images/'

In [ ]:
# Load the dataset
dataset = 'dslr' #@param ["amazon", "webcam", "dslr"]
dataset_folder=MainFolder+dataset+'/images/'
x=[]
y=[]
classes=['back_pack','desk_lamp','mobile_phone','printer','stapler','bike','desktop_computer','monitor','projector','tape_dispenser','bike_helmet',
           'file_cabinet','mouse','punchers','trash_can','bookcase','headphones','mug','ring_binder','bottle','keyboard','paper_notebook','ruler','calculator',
           'laptop_computer','pen','scissors','desk_chair','letter_tray','phone','speaker']
c_discretized=0
for c in classes:
  data = os.listdir(dataset_folder+c)
  for image in data:
    x.append(cv2.resize(cv2.imread(dataset_folder+c+'/'+image), (224,224))/255.0)
    y.append(c_discretized)
  c_discretized+=1
x=np.array(x)
y=(np.array(y))

# Train From Scratch 

In [ ]:
def AlexNetModel():
  # Create AlexNet architecture
  AlexNet=Sequential()

  #1st Convolutional Layer
  AlexNet.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='same', name='conv2d'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

  #2nd Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same', name='Conv2d_1'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

  #3rd Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', name='Conv2d_2'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #4th Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', name='Conv2d_3'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #5th Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', name='Conv2d_4'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

  #Passing it to a Fully Connected layer
  AlexNet.add(Flatten())
  # 1st Fully Connected Layer
  AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  # Add Dropout to prevent overfitting
  AlexNet.add(Dropout(0.4))

  #2nd Fully Connected Layer
  AlexNet.add(Dense(4096))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #3rd Fully Connected Layer
  AlexNet.add(Dense(1000))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #Output Layer
  AlexNet.add(Dense(len(classes)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('softmax'))

  return AlexNet

In [ ]:
# To compare our work, the SOTA repeat 5-fold 2 times
# create k-fold with k=5
kfold = RepeatedKFold(n_splits=5, n_repeats=2)
results_acc=[]
results_val=[]
cont=1

for train, test in kfold.split(x):
  tensorflow.keras.backend.clear_session()
  print('-------------------------------------------------------------------------------------')
  print('Split:',cont)
  print('-------------------------------------------------------------------------------------')
  cont+=1
  #get the new model
  model=AlexNetModel()
  # callback to save the best models
  checkpoint_filepath = 'drive/MyDrive/Multi-Source-Experiments/Train_From_Scratch/'+dataset+'/checkpoint_'+str(cont)+'.h5'
  model_checkpoint=tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
  # compile, fit the model and save the model
  model.compile(loss = keras.losses.categorical_crossentropy, optimizer= tensorflow.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), metrics=['accuracy'])
  history=model.fit(x[train] , to_categorical(y[train]), validation_data=(x[test], to_categorical(y[test])), epochs = 100, callbacks=[model_checkpoint], batch_size=32)
  results_acc.append((history.history['accuracy']))
  results_val.append((history.history['val_accuracy']))

-------------------------------------------------------------------------------------
Split: 1
-------------------------------------------------------------------------------------
Epoch 1/100
20/20 [==============================] - 34s 1s/step - loss: 3.7396 - accuracy: 0.0487 - val_loss: 3.5736 - val_accuracy: 0.0314
Epoch 2/100
20/20 [==============================] - 3s 134ms/step - loss: 2.9950 - accuracy: 0.1792 - val_loss: 4.4172 - val_accuracy: 0.0189
Epoch 3/100
20/20 [==============================] - 5s 278ms/step - loss: 2.5624 - accuracy: 0.2972 - val_loss: 4.0919 - val_accuracy: 0.0566
Epoch 4/100
20/20 [==============================] - 6s 288ms/step - loss: 2.2480 - accuracy: 0.4072 - val_loss: 3.4678 - val_accuracy: 0.0692
Epoch 5/100
20/20 [==============================] - 13s 673ms/step - loss: 2.0085 - accuracy: 0.5440 - val_loss: 3.1697 - val_accuracy: 0.1384
Epoch 6/100
20/20 [==============================] - 15s 790ms/step - loss: 1.8858 - accuracy: 0.5770 - v

In [ ]:
#for split in range(len(results_val)):
print('Train accuracy')
print(np.amax(np.array(results_acc), axis=1))
print('Test accuracy')
print(np.amax(np.array(results_val), axis=1))
np.save('drive/MyDrive/Multi-Source-Experiments/Train_From_Scratch/'+dataset+'/'+dataset+'_train_log.npy', np.array(results_acc))
np.save('drive/MyDrive/Multi-Source-Experiments/Train_From_Scratch/'+dataset+'/'+dataset+'_test_log.npy', np.array(results_val))

# Selective Kernel Transfer

In [ ]:
# some utilities to find the kernel entropies
#--------------------------------------------------------------------------------------------------
# get the distribution of a kernel
def get_distribution(output_list):
  #Get the histogram of the output in a hidden layer
  if np.amax(output_list)==0:
    x=np.histogram(output_list, np.arange(101)/100.0)
    return x[0]
  else:
    x=np.histogram(output_list/np.amax(output_list), np.arange(101)/100.0)
    return x[0]/float(np.sum(x[0]))
#--------------------------------------------------------------------------------------------------
# Find the entropy of a distribution
def distribution_entropy(distribution):
  entropy=0.0
  for dim in distribution:
    if dim!=0:
      entropy-=(dim*math.log2(dim))
  return entropy
#--------------------------------------------------------------------------------------------------
# Get the output of a convolutional layer
def get_output(layer_name, x_set, model):
  #Get the model for an output layer
  output_layer = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
  #return the output of the layer
  return output_layer.predict(np.array(x_set))

#--------------------------------------------------------------------------------------------------
# Find the entropy of each kernel
# The shape of the tensor is 4-dimensional, example: (680, 112, 112, 64)
# each element correspond to: (680)->number of instances, 
#                             (112,112)->shape of the output
#                             (64)-> number of kernels
# so, the information of the kernels is in the last position

def get_kernel_entropy(output_list):
  entropies=[]
  for i in range(output_list.shape[-1]):
    entropies.append(distribution_entropy(get_distribution(output_list[:,:,:,i])))
  return entropies
#--------------------------------------------------------------------------------------------------
# get the kernels to transfer
def obtain_k_to_transfer(x, model_path):
  k_to_transfer=[]
  model=keras.models.load_model(model_path)
  new_weigths=[]
  past=0
  for block in ['conv2d','Conv2d_1','Conv2d_2','Conv2d_3','Conv2d_4']:
    entropies=get_kernel_entropy(get_output(block,list(x),model))
    sorted_entropies_index=sorted(range(len(entropies)), key=lambda k: entropies[k])[:int(len(entropies)/2)]
    if past==0:
      new_weigths.append([model.get_layer(block).get_weights()[0][:,:,:,np.sort(sorted_entropies_index)],model.get_layer(block).get_weights()[1][np.sort(sorted_entropies_index)]])
    else:
      new_weigths.append([model.get_layer(block).get_weights()[0][:,:,np.sort(past)][:,:,:,np.sort(sorted_entropies_index)],
                         model.get_layer(block).get_weights()[1][np.sort(sorted_entropies_index)]])
    past=sorted_entropies_index
  return new_weigths

In [ ]:
def AlexNetModel_50():
  # Create AlexNet architecture
  AlexNet=Sequential()

  #1st Convolutional Layer
  AlexNet.add(Conv2D(filters=48, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='same', name='Conv2d'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

  #2nd Convolutional Layer
  AlexNet.add(Conv2D(filters=128, kernel_size=(5, 5), strides=(1,1), padding='same', name='Conv2d_1'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

  #3rd Convolutional Layer
  AlexNet.add(Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same', name='Conv2d_2'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #4th Convolutional Layer
  AlexNet.add(Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same', name='Conv2d_3'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #5th Convolutional Layer
  AlexNet.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', name='Conv2d_4'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

  #Passing it to a Fully Connected layer
  AlexNet.add(Flatten())
  # 1st Fully Connected Layer
  AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  # Add Dropout to prevent overfitting
  AlexNet.add(Dropout(0.4))

  #2nd Fully Connected Layer
  AlexNet.add(Dense(4096))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #3rd Fully Connected Layer
  AlexNet.add(Dense(1000))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #Output Layer
  AlexNet.add(Dense(len(classes)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('softmax'))

  return AlexNet

In [ ]:
def experiment():
  # To compare our work, the SOTA repeat 5-fold 2 times
  # create k-fold with k=5
  kfold = RepeatedKFold(n_splits=5, n_repeats=2)
  results_acc=[]
  results_val=[]
  cont_g=1

  for train, test in kfold.split(x):
    tensorflow.keras.backend.clear_session()
    print('-------------------------------------------------------------------------------------')
    print('Split:',cont_g)
    print('-------------------------------------------------------------------------------------')
    
    #get the new model
    #model=AlexNetModel()
    # callback to save the best models
    #checkpoint_filepath = 'drive/MyDrive/Multi-Source-Experiments/Selective_Transfer/'+dataset+'/checkpoint_'+str(cont)+'.h5'
    n_w=obtain_k_to_transfer(x, '/content/drive/MyDrive/Multi-Source-Experiments/Train_From_Scratch/webcam/checkpoint_6.h5')
    cont=0
    model=AlexNetModel_50()
    for block in ['Conv2d','Conv2d_1','Conv2d_2','Conv2d_3','Conv2d_4']:
      model.get_layer(block).set_weights(n_w[cont])
      cont+=1
    #model_checkpoint=tensorflow.keras.callbacks.ModelCheckpoint(
    #  filepath=checkpoint_filepath,
    #  save_weights_only=False,
    #  monitor='val_accuracy',
    #  mode='max',
    #  save_best_only=True)
    # compile, fit the model and save the model
    model.compile(loss = keras.losses.categorical_crossentropy, optimizer= tensorflow.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9), metrics=['accuracy'])
    #history=model.fit(x[train] , to_categorical(y[train]), validation_data=(x[test], to_categorical(y[test])), epochs = 100, callbacks=[model_checkpoint], batch_size=32)
    history=model.fit(x[train] , to_categorical(y[train]), validation_data=(x[test], to_categorical(y[test])), epochs = 100, batch_size=32)
    results_acc.append((history.history['accuracy']))
    results_val.append((history.history['val_accuracy']))
    #for split in range(len(results_val)):
  print('Train accuracy')
  print(np.amax(np.array(results_acc), axis=1))
  print('Test accuracy')
  print(np.amax(np.array(results_val), axis=1))
  np.save('drive/MyDrive/Multi-Source-Experiments/Selective_Transfer/'+dataset+'/'+dataset+'_train_log.npy', np.array(results_acc))
  np.save('drive/MyDrive/Multi-Source-Experiments/Selective_Transfer/'+dataset+'/'+dataset+'_test_log.npy', np.array(results_val))
  cont_g+=1
experiment()

In [ ]:
#for split in range(len(results_val)):
print('Train accuracy')
print(np.amax(np.array(results_acc), axis=1))
print('Test accuracy')
print(np.amax(np.array(results_val), axis=1))
np.save('drive/MyDrive/Multi-Source-Experiments/Selective_Transfer/'+dataset+'/'+dataset+'_train_log.npy', np.array(results_acc))
np.save('drive/MyDrive/Multi-Source-Experiments/Selective_Transfer/'+dataset+'/'+dataset+'_test_log.npy', np.array(results_val))

# Multi Source Transfer Learning

In [ ]:
from keras.utils.vis_utils import plot_model
classes=range(31)
def Multi_Source(nw_b1, nw_b2):
  #Multisource transfer version for multisource transfer
  #Alex_Net_M=Sequential()
  inputs=Input(shape=(224,224,3))

  #------------------------------------------------------------------------------------
  #Branch_1
  #------------------------------------------------------------------------------------
  #1st Convolutional Layer
  b1=Conv2D(filters=48, kernel_size=(11,11), strides=(4,4), padding='same', name='b1_conv2d')(inputs)
  b1=BatchNormalization()(b1)
  b1=Activation('relu')(b1)
  b1=MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(b1)

  #2nd Convolutional Layer
  b1=Conv2D(filters=128, kernel_size=(5, 5), strides=(1,1), padding='same', name='b1_Conv2d_1')(b1)
  b1=BatchNormalization()(b1)
  b1=Activation('relu')(b1)
  b1=MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(b1)

  #3rd Convolutional Layer
  b1=Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same', name='b1_Conv2d_2')(b1)
  b1=BatchNormalization()(b1)
  b1=Activation('relu')(b1)

  #4th Convolutional Layer
  b1=Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same', name='b1_Conv2d_3')(b1)
  b1=BatchNormalization()(b1)
  b1=Activation('relu')(b1)

  #5th Convolutional Layer
  b1=Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', name='b1_Conv2d_4')(b1)
  b1=BatchNormalization()(b1)
  b1=Activation('relu')(b1)
  b1=MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(b1)

  #------------------------------------------------------------------------------------
  #Branch_2
  #------------------------------------------------------------------------------------
  b2=Conv2D(filters=48, kernel_size=(11,11), strides=(4,4), padding='same', name='b2_conv2d')(inputs)
  b2=BatchNormalization()(b2)
  b2=Activation('relu')(b2)
  b2=MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(b2)

  #2nd Convolutional Layer
  b2=Conv2D(filters=128, kernel_size=(5, 5), strides=(1,1), padding='same', name='b2_Conv2d_1')(b2)
  b2=BatchNormalization()(b2)
  b2=Activation('relu')(b2)
  b2=MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(b2)

  #3rd Convolutional Layer
  b2=Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same', name='b2_Conv2d_2')(b2)
  b2=BatchNormalization()(b2)
  b2=Activation('relu')(b2)

  #4th Convolutional Layer
  b2=Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding='same', name='b2_Conv2d_3')(b2)
  b2=BatchNormalization()(b2)
  b2=Activation('relu')(b2)

  #5th Convolutional Layer
  b2=Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='same', name='b2_Conv2d_4')(b2)
  b2=BatchNormalization()(b2)
  b2=Activation('relu')(b2)
  b2=MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(b2)
  join=concatenate([b1,b2])
  #Passing it to a Fully Connected layer
  join=Flatten()(join)
  # 1st Fully Connected Layer
  join=Dense(4096, input_shape=(32,32,3,))(join)
  join=BatchNormalization()(join)
  join=Activation('relu')(join)
  # Add Dropout to prevent overfitting
  join=Dropout(0.4)(join)

  #2nd Fully Connected Layer
  join=Dense(4096)(join)
  join=BatchNormalization()(join)
  join=Activation('relu')(join)
  #Add Dropout
  join=Dropout(0.4)(join)

  #3rd Fully Connected Layer
  join=Dense(1000)(join)
  join=BatchNormalization()(join)
  join=Activation('relu')(join)
  #Add Dropout
  join=Dropout(0.4)(join)

  #Output Layer
  join=Dense(len(classes))(join)
  join=BatchNormalization()(join)
  join=Activation('softmax')(join)


  Alex_Net_M= Model(inputs, join)
  cont=0
  for block in ['conv2d','Conv2d_1','Conv2d_2','Conv2d_3','Conv2d_4']:
    Alex_Net_M.get_layer('b1_'+block).set_weights(nw_b1[cont])
    Alex_Net_M.get_layer('b2_'+block).set_weights(nw_b2[cont])
    cont+=1


branch_1=obtain_k_to_transfer(x, '/content/drive/MyDrive/Multi-Source-Experiments/Train_From_Scratch/webcam/checkpoint_6.h5')
branch_2=obtain_k_to_transfer(x, '/content/drive/MyDrive/Multi-Source-Experiments/Train_From_Scratch/webcam/checkpoint_6.h5')
Multi_Source(branch_1, branch_2)